# Load various df

Load item_code and premise_code data. Item_code and premise_code are fixed for all months

In [1]:
import numpy as np
import pandas as pd

URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_premise.parquet'
df_premise_code = pd.read_parquet(URL_DATA)
if 'date' in df_premise_code.columns: df_premise_code['date'] = pd.to_datetime(df_premise_code['date'])


URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_item.parquet'
df_item_code = pd.read_parquet(URL_DATA)
if 'date' in df_item_code.columns: df_item_code['date'] = pd.to_datetime(df_item_code['date'])



Load price data

In [2]:
def merge_data(df):

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning

    print(tempdf.head())
    #
    p =tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')["premise_type"])
    p.name = 'premise_type'


    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    q =tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')["premise"])
    q.name = 'premise'

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    s =tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')["state"])
    s.name = 'state'

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    t = tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')['district'])
    t.name = "district"

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    u = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['item'])
    u.name = "item"

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    v = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['unit'])
    v.name = "unit"

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    w = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['item_category'])
    w.name = "item_category"

    
    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    w2 = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['item_group'])
    w2.name = "item_group"

    #print('S:\n',s) 
    #print('T:\n',t)

    df_merged = pd.concat([df,p,q,s,t,u,v,w,w2],axis=1)
    return df_merged    

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime


#Get all months from Jan 23 until today
datelist = pd.date_range(start ='05/01/2024' , end = '06/30/2024',freq="M").tolist() #be careful. dates are mm/dd/yyyy
datelist = [x.strftime('%Y-%m') for x in datelist]
print(datelist)

def get_all_dataframes(datelist):
    
    df_list=[]

    for month_d in datelist:

        month_parq = month_d +'.parquet'
        #URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/pricecatcher_{}'.format(month_parq)
        URL_DATA = 'https://storage.data.gov.my/pricecatcher/pricecatcher_{}'.format(month_parq)


        print(URL_DATA)
        df = pd.read_parquet(URL_DATA)
        if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])
        #df.head()
        df_list.append(df)
    
    return df_list

#This stores all price dataframes since Jan 22
df_all_list = get_all_dataframes(datelist)
print(f'Number of dataframes in the list: {len(df_all_list)}')


['2024-05', '2024-06']
https://storage.data.gov.my/pricecatcher/pricecatcher_2024-05.parquet
https://storage.data.gov.my/pricecatcher/pricecatcher_2024-06.parquet
Number of dataframes in the list: 2


In [5]:
for i, df in enumerate(df_all_list):
    df_all_list[i] = merge_data(df_all_list[i])


        date  premise_code  item_code  price
0 2024-05-01         20908        190    5.9
1 2024-05-01         20908        191   10.9
2 2024-05-01         20908        350    6.9
3 2024-05-01         20908        360    1.4
4 2024-05-01         20908        364    8.8
        date  premise_code  item_code  price
0 2024-06-01             2          1   10.8
1 2024-06-01             2          9   36.0
2 2024-06-01             2         14   26.0
3 2024-06-01             2         16    4.0
4 2024-06-01             2         18    6.0


Merge price and premise info

In [6]:
df_master = pd.DataFrame(columns=['date','premise_code','item_code','price','premise_type','premise','state','district','item','unit','item_category'])
#df_master.head()
for df in df_all_list:
    df_master = pd.concat([df_master,df])

df_master.head()    

,date,premise_code,item_code,price,premise_type,premise,state,district,item,unit,item_category,item_group
0,2024-05-01,20908,190,5.9,NaN,NaN,NaN,NaN,SARDIN CAP AYAM (SOS TOMATO),155 g,IKAN DALAM TIN,BARANGAN BERBUNGKUS
1,2024-05-01,20908,191,10.9,NaN,NaN,NaN,NaN,SARDIN CAP AYAM (SOS TOMATO),425 g,IKAN DALAM TIN,BARANGAN BERBUNGKUS
2,2024-05-01,20908,350,6.9,NaN,NaN,NaN,NaN,SERBUK KARI IKAN BABAS,250 g,REMPAH RATUS (BERBUNGKUS),BARANGAN BERBUNGKUS
3,2024-05-01,20908,360,1.4,NaN,NaN,NaN,NaN,SUP BUNJUT ADABI,8 g,REMPAH RATUS (BERBUNGKUS),BARANGAN BERBUNGKUS
4,2024-05-01,20908,364,8.8,NaN,NaN,NaN,NaN,SERBUK CILI BABAS,250 g,REMPAH RATUS (BERBUNGKUS),BARANGAN BERBUNGKUS


Add Monthdate

In [7]:
df_master['monthdate'] = df_master["date"].apply(lambda x: x.strftime('%Y-%m'))

# Master Dataframe

In [8]:
df_master.sample(5)

,date,premise_code,item_code,price,premise_type,premise,state,district,item,unit,item_category,item_group,monthdate
2227843,2024-05-28,11571,1899,17.99,Pasar Raya / Supermarket,"HERO MARKET (SEKSYEN 6, WANGSA MAJU )",W.P. Kuala Lumpur,Setiawangsa,CILI KERING KERINTING (BERTANGKAI/TIDAK BERTAN...,1kg,CILI KERING,BARANGAN KERING,2024-05
1131491,2024-05-15,11956,108,10.00,Pasar Basah,PASAR PADANG TEMBAK,Pulau Pinang,Timur Laut,KUNYIT HIDUP,1kg,SAYUR-SAYURAN,BARANGAN SEGAR,2024-05
857778,2024-06-13,1942,350,6.50,Kedai Runcit,LIM GUAN HUAT,Sarawak,Song,SERBUK KARI IKAN BABAS,250 g,REMPAH RATUS (BERBUNGKUS),BARANGAN BERBUNGKUS,2024-06
44631,2024-05-02,17778,1818,15.99,Pasar Raya / Supermarket,SHUANG-LIM CASH & CARRY,W.P. Kuala Lumpur,Wangsa Maju,KACANG BOTOL,1kg,SAYUR-SAYURAN,BARANGAN SEGAR,2024-05
2466715,2024-05-30,17990,272,3.00,Pasar Raya / Supermarket,PASARAYA BS TEMERLOH SDN BHD,Pahang,Temerloh,ROTI SANDWICH GARDENIA ORIGINAL CLASSIC,400 g,ROTI,BARANGAN BERBUNGKUS,2024-05


In [9]:
df_master['item_group'].unique()

array(['BARANGAN BERBUNGKUS', nan, 'BARANGAN SEGAR', 'BARANGAN KERING',
       'SUSU DAN BARANGAN BAYI', 'MINUMAN', 'PRODUK KEBERSIHAN'],
      dtype=object)

# Df barangan segar

In [10]:
df_barangan_segar = df_master[df_master['item_group']=='BARANGAN SEGAR']

In [11]:
print(df_barangan_segar.shape)
print(f'{df_barangan_segar.item_code.nunique()} unique items for may and june 2024')
df_barangan_segar.head()

(2606131, 13)
156 unique items for may and june 2024


,date,premise_code,item_code,price,premise_type,premise,state,district,item,unit,item_category,item_group,monthdate
14,2024-05-02,2,1,10.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,AYAM BERSIH - STANDARD,1kg,AYAM,BARANGAN SEGAR,2024-05
15,2024-05-02,2,9,36.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...,1kg,DAGING,BARANGAN SEGAR,2024-05
16,2024-05-02,2,16,4.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,BETIK BIASA,1kg,BUAH-BUAHAN,BARANGAN SEGAR,2024-05
17,2024-05-02,2,18,6.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,PISANG BERANGAN,1kg,BUAH-BUAHAN,BARANGAN SEGAR,2024-05
18,2024-05-02,2,19,6.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,PISANG EMAS,1kg,BUAH-BUAHAN,BARANGAN SEGAR,2024-05


# Price Change Master_df

In [ ]:
#Select item i.e ayam
#Select monthdate i.e aug 23
#compute max,min, mean

#df_one_item = df_master[df_master["item_code"]==131]

df_price_change_ALLitems = []

#Extract item_code and monthdate data
item_c_list  = df_barangan_segar["item_code"].unique() #take the list of items from the latest price dataframe



mthdate_list = df_barangan_segar["monthdate"].unique() #take the list of monthdates from df_master as this a combined df since jan22

i=1
for item_c in item_c_list:

    #Select ONE item
    x_df = df_barangan_segar[df_barangan_segar["item_code"]==item_c]
    x_index = x_df.index


    #Get ONE item details
    item = x_df["item"].unique()
    item = item[0]
    unit = x_df["unit"].unique()
    unit = unit[0]

    #print(f'{i}. item:{item} unit:{unit}')
    
    
    for mthdate in mthdate_list:
        
        #Select ONE monthdate
        y_df = x_df[x_df["monthdate"]==mthdate]
        
        if y_df.empty:
            #print('ONE ITEM DF IS EMPTY~~')
            print(f'ONE ITEM DF IS EMPTY!!~~ {i}. item_c:{item_c} item:{item} unit:{unit} mthdate:{mthdate}')

        else:

            #print(f'{i}. item:{item} unit:{unit} mthdate:{mthdate}')
            print(i)

            max_y = y_df["price"].max()
            min_y = y_df["price"].min()
            mean_y = y_df["price"].mean()

            df_price_change_ALLitems.append([item_c,item,unit,mthdate,max_y,min_y,mean_y])
    i+=1

    
    
print(len(x_df))
x_df.head()

df_price_change_ALLitems = pd.DataFrame(df_price_change_ALLitems,columns=['item_code','item','unit','monthdate','max','min','mean'])
df_price_change_ALLitems.head()


In [ ]:
df_price_change_ALLitems['item_code'].nunique()

In [ ]:
#df_price_change_ALLitems.to_csv('df_price_change_ALLitems.csv')

In [ ]:
len(df_price_change_ALLitems)

In [ ]:
df_price_change_ALLitems['item_code'].unique()

In [ ]:
#cili kering 1899,1900
#bawang putih 1564
#ayam 1 kg 1
#bawang merah 131,132,1441,1442,1933

i = 1

#df_plot = df_price_change_ALLitems[df_price_change_ALLitems["item_code"]==item_c_list[i]]
df_plot = df_price_change_ALLitems[df_price_change_ALLitems["item_code"]==i]

item_name = df_plot["item"].unique()
item_name = item_name[0]

item_unit = df_plot["unit"].unique()
item_unit = item_unit[0]

title_plot = item_name + ' ' + item_unit

#df_plot.plot.scatter(x="monthdate",y="mean", title=title_plot, rot=90)
df_plot.plot.scatter(x="monthdate",y="mean", title=title_plot, rot=90)




In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rc('xtick', labelsize=5) 
matplotlib.rc('ytick', labelsize=5) 
fig, axes = plt.subplots(nrows=13, ncols=13)

i=0
j=0

item_count=1
print(len(item_c_list))
for w,item_c in enumerate(item_c_list):

    #print(f'{w} {item_c}')

    #print(f'item_count:{item_count} i:{i} j:{j}')

    df_plot = df_price_change_ALLitems[df_price_change_ALLitems["item_code"]==item_c]

    item_name = df_plot["item"].unique()
    item_name = item_name[0]

    item_unit = df_plot["unit"].unique()
    item_unit = item_unit[0]

    #title_plot = item_name + ' ' + item_unit
    title_plot = item_name
    

    #df_plot.plot.scatter(x="monthdate",y="mean", title=title_plot, rot=90, ax=axes[i,j])
    df_plot.plot.line(x="monthdate",y="mean", title=title_plot, ax=axes[i,j], rot=45,figsize=(30,30),legend=False)

    j+=1

    if (j==13):
        j=0
        i+=1
   

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rc('xtick', labelsize=10) 
matplotlib.rc('ytick', labelsize=10) 
fig, axes = plt.subplots(nrows=7, ncols=6)

i=0
j=0

item_count=1
print(len(item_c_list))
for w,item_c in enumerate(item_c_list):

    #print(f'{w} {item_c}')

    #print(f'item_count:{item_count} i:{i} j:{j}')

    df_plot = df_price_change_ALLitems[df_price_change_ALLitems["item_code"]==item_c]

    item_name = df_plot["item"].unique()
    item_name = item_name[0]

    item_unit = df_plot["unit"].unique()
    item_unit = item_unit[0]

    #title_plot = item_name + ' ' + item_unit
    title_plot = item_name
    

    #df_plot.plot.scatter(x="monthdate",y="mean", title=title_plot, rot=90, ax=axes[i,j])
    ax = df_plot.plot.scatter(x="monthdate",y="mean", ax=axes[i,j], rot=45,figsize=(30,30),label=title_plot)
    ax.legend(loc='upper center')
    

    j+=1

    if (j==6):
        j=0
        i+=1
    
    if (w==40):
        break

# Previous Analysis on LinkedIn

![Alt text](image-5.png)

# Linear regression

## Regression on ONE item

In [ ]:
from sklearn import linear_model
data = df_price_change_ALLitems[df_price_change_ALLitems['item_code']==1]

x = np.arange(1,10) #for jan to september because current monthdate is in string

x_train = np.array(x.reshape(-1, 1))
y_train = np.array(data['mean']).reshape(-1, 1)

model = linear_model.LinearRegression()
model.fit(x_train,y_train)

r2_score = model.score(x_train,y_train)
print(f'R2_squared: {r2_score}')


print('coeff',model.coef_)
print('intercept',model.intercept_)


y_pred = model.predict(x_train)
plt.scatter(x_train, y_train, color ='b')
plt.plot(x_train, y_pred, color ='k')
 
plt.show()

## Regression for all items

In [ ]:
from sklearn import linear_model

import matplotlib
import matplotlib.pyplot as plt


x = np.arange(1,10) #for jan to september because current monthdate is in string
x_train = np.array(x.reshape(-1, 1))

row_lim = 3
col_lim = 3
fig, axes = plt.subplots(nrows=row_lim, ncols=col_lim,figsize=(30,20))


i=0
j=0
for w, item_c in enumerate(item_c_list):

    #print(f'i:{i} j:{j}')

    data = df_price_change_ALLitems[df_price_change_ALLitems['item_code']==item_c]
    item_name = data["item"].unique()
    item_name = item_name[0]

    item_unit = data["unit"].unique()
    item_unit = item_unit[0]

    #title_plot = item_name + ' ' + item_unit
    title_plot = item_name


    y_train = np.array(data['mean']).reshape(-1, 1)

    model = linear_model.LinearRegression()
    model.fit(x_train,y_train)

    y_pred = model.predict(x_train)
    

    axes[i,j].scatter(x_train, y_train, color ='b')
    axes[i,j].plot(x_train, y_pred, color ='k')
    axes[i,j].tick_params(axis='both', which='major', labelsize=10)
    axes[i,j].set_title(title_plot,fontsize=15)
    


    j+=1

    if (j==col_lim):
        j=0
        i+=1
    
    if (i==row_lim):
        break


for ax in axes.flat:
    #ax.set(xlabel='Months', ylabel='Mean price RM')
    ax.set_xlabel('Bulan',fontsize=18)
    ax.set_ylabel('Harga Purata RM ',fontsize=18)

# Hide x labels and tick labels for top plots and y ticks for right plots.
#for ax in axes.flat:
#    ax.label_outer()
